In [ ]:
from anthropic import Anthropic

client = Anthropic()

In [ ]:
# TODO: use and improve these tools
# hint: json schemas support these types: string, number, boolean, object, array
import math


def get_stock_price(ticker: str):
    stocks = {
        "aapl": 195.50,    # Apple
        "msft": 425.30,    # Microsoft
        "nvda": 875.20,    # NVIDIA
        "goog": 162.75,    # Alphabet (Google)
        "googl": 162.75,    # Alphabet (Google)
        "amzn": 185.40,    # Amazon
        "meta": 520.80,    # Meta (Facebook)
        "tsla": 245.60,    # Tesla
        # who cares about the other stocks anyway?
    }
    return stocks[ticker]


get_stock_price_spec = {
    "name": "get_stock_price",
    "description": "There is a list of given companies and their stock prices, in the comments you have the full name of the company.",
    "input_schema": {
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Return the name of the listing name for example Tesla is tsla.",
            },
        },
        "required": ["ticker"],
    },
}


def calculate(op: str, input1: float, input2: float):
    match op:
        case "+":
            return input1 + input2
        case "-":
            return input1 - input2
        case "*":
            return input1 * input2
        case "/":
            return input1 / input2
        case "**":
            return input1 ** input2
        case "log":
            return math.log(input1, input2)

calculate_spec = {
    "name": "calculate",
    "description": "calculator",
    "input_schema": {
        "type": "object",
        "properties": {
            "op": {
                "type": "string",
                "description": "operator",
            },
            "input1": {
                "type": "number",
                "description": "input1"
            },
            "input2": {
                "type": "number",
                "description": "input2"
            },
        },
        "required": ["op", "input1", "input2"],
    },
}


def call_claude(messages: list[dict], tools: list[dict] = [], system_prompt="") -> dict:
    """Helper function to hit the API for you with tool use.
    full docs: https://docs.anthropic.com/en/docs/build-with-claude/tool-use/overview#single-tool-example"""
    return client.messages.create(
        model="claude-3-5-haiku-20241022",  # NOTE: for this interview, we'll only use haiku
        max_tokens=1024,
        temperature=0.0,
        system=system_prompt,
        tools=tools,
        messages=messages,
    )

In [ ]:
calculate("*", 3232,173)

In [ ]:

def example():
    """This code should work for a query that can be answered in a single tool call, but wouldn't handle anything beyond that."""
    prompt = "Use the stock tool to look up the 'tsla' ticker"

    # 1. Passing initial prompt to claude
    messages = [{"role": "user", "content": prompt}]
    response = call_claude(
        messages,
        tools=[get_stock_price_spec, calculate_spec],
        system_prompt=system,
    )
    # append claude's response into the messages thread
    messages.append({"role": "assistant", "content": response.content})

    # 2. Extract the tool call and execute the tool.
    # Here we assume that the get_stock_price tool was called in the last content block. This is not guaranteed!
    # ANY content block can be a tool call, but for this example prompt, claude usually responds with text first, and then a single tool call
    tool_call = response.content[-1]
    tool_output = get_stock_price(
        tool_call.input["ticker"]
    )  # tool_call.input contains a dict with the tool call parameters
    # append our tool result into the messages thread. Note tool_call.id
    messages.append(
        {
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": tool_call.id,
                    "content": str(tool_output),
                }
            ],
        }
    )

    # 3. Passing the tool call results back to claude:
    response2 = call_claude(messages, tools=[get_stock_price_spec, calculate_spec])
    print(response2)


example()

In [ ]:
# Example code showing a **single step** of doing a tool call and passing the result back to claude
system = """
        You are an math expert with 15+ years of teaching and also have knowledge about the finanacial world. 
        You proceed with great care after analyzing board mass rule.
        You plan each step carefully and then proceed.
"""

In [ ]:
def get_tool_ouput(block, messages, tool_call):
    if block.name == "get_stock_price":
                tool_output = get_stock_price(
                    tool_call.input["ticker"].lower()
                )  # tool_call.input contains a dict with the tool call parameters
                # append our tool result into the messages thread. Note tool_call.id

    else:
        inputs = block.input
        tool_output = calculate(
            inputs["op"], inputs["input1"], inputs["input1"]
        )
        print("Calc tool output", tool_output)

    messages.append(
        {
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": tool_call.id,
                    "content": str(tool_output),
                }
            ],
        }
    )
    
    return messages

In [64]:
# TODO: this is where you'll implement your agent loop in the finance_agent function
# Your goal is to make an agent function that can successfully answer each of the prompts below.


def finance_agent(prompt: str):
    """
    TODO: this is where you will complete the "agent loop".
    Look at example() defined in the cell above for some starter code of using the anthropic API with tools
    and how to handle the response objects.
    """
    ...
    messages = [{"role": "user", "content": prompt}]
    response = call_claude(messages, tools=[get_stock_price_spec, calculate_spec])
    print("Initial Response :",response.content)

    while response.stop_reason != "end_turn":
        for block in response.content:
            if block.type == "tool_use":
                messages.append({"role": "assistant", "content": response.content})

                tool_call = response.content[-1]
                if block.name == "get_stock_price":
                    tool_output = get_stock_price(
                        tool_call.input["ticker"]
                    )  # tool_call.input contains a dict with the tool call parameters
                    # append our tool result into the messages thread. Note tool_call.id

                else:
                    inputs = block.input
                    print("in calc")
                    print("op",inputs["op"],"input1", inputs["input1"],"input2", inputs["input2"])
                    tool_output = calculate(
                        inputs["op"], inputs["input1"], inputs["input2"]
                    )
                    print("Calc tool output", tool_output)

                messages.append(
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "tool_result",
                                "tool_use_id": tool_call.id,
                                "content": str(tool_output),
                            }
                        ],
                    }
                )

                response = call_claude(
                    messages, tools=[get_stock_price_spec, calculate_spec]
                )
                print("Inside tool :",response.content)

            elif block.type == "text":
                print(response.content[0].text)


######################################################
# Here are some example queries that we'd like to work

prompt0 = "Use the stock tool to look up the 'tsla' ticker"
# answer0 = 245.60

prompt1 = "What's Google's stock price?"
# answer1 = 245.60

prompt2 = "Help me solve this math problem: 173 * 3232 + 342 / 72.1"
# answer2 = 559,140.74341

prompt3 = "Help me solve this math problem: sqrt(234.13) + ln(27389140.25) + 173 * 32 + 4.5^2."
# answer3 = 5588.677

prompt4 = "How much would it cost to buy 15 shares of tesla, 24 shares of google, and 120 shares of amazon?"
# answer4 = 29,838

prompt5 = "Hi, what capabilities do you have?"
# answer5 = mentions being able to check stock prices and do calculations

# finance_agent(prompt0)
# finance_agent(prompt1)
# finance_agent(prompt2)
# finance_agent(prompt3)
finance_agent(prompt4)
# finance_agent(prompt5)

Initial Response : [TextBlock(citations=None, text="I'll help you calculate the total cost by first getting the current stock prices for these companies and then multiplying by the number of shares.\n\nLet's check the stock prices:\n\n1. Tesla (TSLA):", type='text'), ToolUseBlock(id='toolu_01Fvgdup1AQLCKx1YwuRdpDu', input={'ticker': 'tsla'}, name='get_stock_price', type='tool_use')]
I'll help you calculate the total cost by first getting the current stock prices for these companies and then multiplying by the number of shares.

Let's check the stock prices:

1. Tesla (TSLA):
Inside tool : [TextBlock(citations=None, text='2. Google (GOOGL):', type='text'), ToolUseBlock(id='toolu_01P9wxi9mwC2AxKGpxPoLzCM', input={'ticker': 'googl'}, name='get_stock_price', type='tool_use')]
2. Google (GOOGL):
Inside tool : [TextBlock(citations=None, text='3. Amazon (AMZN):', type='text'), ToolUseBlock(id='toolu_01HGRpyKsM4UqanWewMzFAUE', input={'ticker': 'amzn'}, name='get_stock_price', type='tool_use')]

In [ ]:
messages = [{"role": "user", "content": prompt3}]
response = call_claude(
    messages, tools=[get_stock_price_spec, calculate_spec], system_prompt=system
)
response

In [ ]:
print(response.stop_reason)

In [ ]:
for block in response.content:
    print(block)
    if block.type == "tool_use":
        if block.name == "calculate":
            print("in tool use")
            inputs= block.input
            print(inputs["op"], inputs["input1"], inputs["input1"])
        elif block.name == "get_stock_price":
            print("in get stock price")

In [ ]:
response.stop_reason